# Python API Homework - What's the Weather Like?

## Background

Whether financial, political, or social -- data's true power lies in its
ability to answer questions definitively. So let's take what you've learned 
about Python requests, APIs, and JSON traversals to answer a fundamental
question: "What's the weather like as we approach the equator?"

Now, we know what you may be thinking: _"Duh. It gets hotter..."_

But, if pressed, how would you **prove** it?

![Equator](Images/equatorsign.png)

### Before You Begin

1. Create a new repository for this project called `python-api-challenge`. 
   **Do not add this homework to an existing repository**.

2. Clone the new repository to your computer.

3. Inside your local git repository, create a directory for both of the
   Python Challenges. Use folder names corresponding to the challenges: **WeatherPy**.

        
4. Inside the folder that you just created, add new files called `WeatherPy.ipynb` 
   and `VacationPy.ipynb`. These will be the main scripts to run for each
   analysis.

5. Push the above changes to GitHub.

*******************************************************


## Part I - WeatherPy

In this example, you'll be creating a Python script to visualize the weather of 
500+ cities across the world of varying distance from the equator. 
To accomplish this, you'll be utilizing a [simple Python library]
(https://pypi.python.org/pypi/citipy), 
the [OpenWeatherMap API](https://openweathermap.org/api), and a little 
common sense to create a representative model of weather across world cities.

Your first requirement is to create a series of scatter plots to showcase 
the following relationships:

* Temperature (F) vs. Latitude
* Humidity (%) vs. Latitude
* Cloudiness (%) vs. Latitude
* Wind Speed (mph) vs. Latitude

After each plot add a sentence or too explaining what the code is and 
analyzing.

Your second requirement is to run linear regression on each relationship, 
only this time separating them into Northern Hemisphere (greater than 
or equal to 0 degrees latitude) and Southern Hemisphere (less than 0 
degrees latitude):

* Northern Hemisphere - Temperature (F) vs. Latitude
* Southern Hemisphere - Temperature (F) vs. Latitude
* Northern Hemisphere - Humidity (%) vs. Latitude
* Southern Hemisphere - Humidity (%) vs. Latitude
* Northern Hemisphere - Cloudiness (%) vs. Latitude
* Southern Hemisphere - Cloudiness (%) vs. Latitude
* Northern Hemisphere - Wind Speed (mph) vs. Latitude
* Southern Hemisphere - Wind Speed (mph) vs. Latitude

After each pair of plots explain what the linear regression is modeling 
such as any relationships you notice and any other analysis you may have.

**Optional** You will be creating multiple linear regression plots. 
To optimize your code, write a function that creates the linear regression plots.

Your final notebook must:

* Randomly select **at least** 500 unique (non-repeat) cities based on 
  latitude and longitude.
* Perform a weather check on each of the cities using a series of successive 
  API calls.
* Include a print log of each city as it's being processed with the city 
  number and city name.
* Save a CSV of all retrieved data and a PNG image for each scatter plot.



# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import urllib
import json
import seaborn as sns
from citipy import citipy
import random
from pprint import pprint
import csv

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [4]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

625

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [5]:
# OpenWeatherMap API Key
api_key = weather_api_key = "0fa2d78562d743fb83f04b84910036ae"

# Starting URL for Weather Map API Call
url = 'http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=' + api_key

url

'http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae'

In [6]:
# I was having some trouble with the appropriate way to perform the api calls so I have two versions thus far!!!!
# Making an empty list to which we will append information obtained from these api calls and the
# output and enter them into lists and then into a DataFrame for further analyses and visualization

city_data = []

# using data provided to us in the starter code, which took some time for me to figure out what to do with

print("Beginning Data Retrieval     ")
print("-----------------------------")

# variables to iterate and use to count items
record_count = 1
set_count = 1

# Iterate over all the cities in our list, [cities], made in the 2nd cell
for i, city in enumerate(cities):
        
    # Group cities in sets of 50 for print log, or logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 0

    # Create an endpoint URL with each city
    city_url = url + "&q=" + urllib.request.pathname2url(city)

    # Log the url, record, and set numbers
    print("Processing Record %s of Set %s | %s" % (record_count, set_count, city))
    print(city_url)

    # Add 1 to the record count for each single additional iteration
    record_count += 1

    # perform an API request for each of the cities
    try:
        # Parse the JSON and retrieve data
        city_weather = requests.get(city_url).json()

        # Parse out the max temp, humidity, and cloudiness
        city_latitute = city_weather["coord"]["lat"]
        city_longitude = city_weather["coord"]["lon"]
        city_max_temperature = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_date = city_weather["dt"]

        # Append the City information into city_data list
        city_data.append({"City": city, 
                          "Lat": city_latitute, 
                          "Lng": city_longitude, 
                          "Max Temp": city_max_temperature,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

    # If an error is experienced, skip the city
    except:
        print("City not found...")
        pass
              
# Indicate that Data Loading is complete 
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | carnarvon
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=carnarvon
City not found...
Processing Record 2 of Set 1 | mount darwin
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=mount%20darwin
City not found...
Processing Record 3 of Set 1 | hamilton
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=hamilton
City not found...
Processing Record 4 of Set 1 | kapaa
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=kapaa
City not found...
Processing Record 5 of Set 1 | peno
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=peno
City not found...
Processing Record 6 of Set 1 | vaini
http://api.openweathermap.org/data/2.5/weather?units=Im

City not found...
Processing Record 49 of Set 1 | camacari
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=camacari
City not found...
Processing Record 50 of Set 1 | leningradskiy
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=leningradskiy
City not found...
Processing Record 0 of Set 2 | saskylakh
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=saskylakh
City not found...
Processing Record 1 of Set 2 | esperance
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=esperance
City not found...
Processing Record 2 of Set 2 | amderma
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=amderma
City not found...
Processing Record 3 of Set 2 | hithadhoo
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d7856

City not found...
Processing Record 49 of Set 2 | bambanglipuro
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=bambanglipuro
City not found...
Processing Record 0 of Set 3 | buinsk
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=buinsk
City not found...
Processing Record 1 of Set 3 | kamenskoye
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=kamenskoye
City not found...
Processing Record 2 of Set 3 | riviere-au-renard
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=riviere-au-renard
City not found...
Processing Record 3 of Set 3 | verkhniy avzyan
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=verkhniy%20avzyan
City not found...
Processing Record 4 of Set 3 | paamiut
http://api.openweathermap.org/data/2.5/weather?u

City not found...
Processing Record 49 of Set 3 | karasjok
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=karasjok
City not found...
Processing Record 0 of Set 4 | butaritari
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=butaritari
City not found...
Processing Record 1 of Set 4 | matay
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=matay
City not found...
Processing Record 2 of Set 4 | inhambane
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=inhambane
City not found...
Processing Record 3 of Set 4 | vostok
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=vostok
City not found...
Processing Record 4 of Set 4 | san patricio
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b8

City not found...
Processing Record 49 of Set 4 | mangan
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=mangan
City not found...
Processing Record 0 of Set 5 | de aar
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=de%20aar
City not found...
Processing Record 1 of Set 5 | upernavik
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=upernavik
City not found...
Processing Record 2 of Set 5 | acapulco
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=acapulco
City not found...
Processing Record 3 of Set 5 | osypenko
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=osypenko
City not found...
Processing Record 4 of Set 5 | luxor
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036

...

City not found...
Processing Record 3 of Set 9 | ajdabiya
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=ajdabiya
City not found...
Processing Record 4 of Set 9 | bagdarin
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=bagdarin
City not found...
Processing Record 5 of Set 9 | benoy
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=benoy
City not found...
Processing Record 6 of Set 9 | quelimane
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=quelimane
City not found...
Processing Record 7 of Set 9 | alice springs
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=alice%20springs
City not found...
Processing Record 8 of Set 9 | cururupu
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb

City not found...
Processing Record 3 of Set 10 | luderitz
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=luderitz
City not found...
Processing Record 4 of Set 10 | valparaiso
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=valparaiso
City not found...
Processing Record 5 of Set 10 | chernyakhovsk
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=chernyakhovsk
City not found...
Processing Record 6 of Set 10 | huarmey
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=huarmey
City not found...
Processing Record 7 of Set 10 | yefira
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=yefira
City not found...
Processing Record 8 of Set 10 | bengkulu
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562

City not found...
Processing Record 2 of Set 11 | nouadhibou
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=nouadhibou
City not found...
Processing Record 3 of Set 11 | krasnoarmeysk
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=krasnoarmeysk
City not found...
Processing Record 4 of Set 11 | garchitorena
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=garchitorena
City not found...
Processing Record 5 of Set 11 | ende
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=ende
City not found...
Processing Record 6 of Set 11 | dien bien
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=dien%20bien
City not found...
Processing Record 7 of Set 11 | naron
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa

City not found...
Processing Record 0 of Set 12 | ekhabi
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=ekhabi
City not found...
Processing Record 1 of Set 12 | ambulu
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=ambulu
City not found...
Processing Record 2 of Set 12 | virginia beach
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=virginia%20beach
City not found...
Processing Record 3 of Set 12 | ust-tsilma
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=ust-tsilma
City not found...
Processing Record 4 of Set 12 | igarka
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=igarka
City not found...
Processing Record 5 of Set 12 | geraldton
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d

City not found...
Processing Record 0 of Set 13 | acarau
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=acarau
City not found...
Processing Record 1 of Set 13 | barreirinhas
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=barreirinhas
City not found...
Processing Record 2 of Set 13 | isiro
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=isiro
City not found...
Processing Record 3 of Set 13 | sesheke
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=sesheke
City not found...
Processing Record 4 of Set 13 | kaseda
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b84910036ae&q=kaseda
City not found...
Processing Record 5 of Set 13 | gorontalo
http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=0fa2d78562d743fb83f04b849

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [7]:
# Convert array of JSONs into Pandas DataFrame
city_data_pd = pd.DataFrame(city_data)




# Show Record Count
city_data_pd.count()

Series([], dtype: int64)

In [11]:
# DataFrame below to display-using different method to create a df than usual
city_data_pd

""


In [12]:
# Export the City_Data into a csv
city_data_pd.to_csv("WeatherPy.csv",encoding="utf-8",index=False)

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [13]:
# First filter out those rows which the rows don't contain any data

city_data_pd_dropna = city_data_pd.dropna(how = 'all') 
city_data_pd_dropna  


""


In [14]:
#  Get the indices of cities that have humidity over 100%.
# Filter all rows for which the cities' humidities are = 100%
city_data_pd_filtered = city_data_pd[city_data_pd['Humidity'] == 100] 
  
# Print the new dataframe 
city_data_pd_filtered 
  


  


KeyError: 'Humidity'

In [ ]:
# Print the shape of the dataframe 
city_data_pd_filtered.shape

In [ ]:
# Filter all rows for which city's have a humidity equal to 100%
city_data_pd.drop(city_data_pd[city_data_pd['Humidity'] == 100].index, inplace = True) 
  
# Print the modified dataframe 
city_data_pd 
  
 

In [ ]:
city_data_pd.shape


In [ ]:
# We went from 567,9 to 550,9-->so 17 cities had humidities = 100%

city_data_pd.shape

In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".

import seaborn as sns
sns.boxplot(x = city_data_pd['Humidity'])

In [ ]:
# Extract relevant fields from the data frame


# Export the City_Data into a csv


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

In [ ]:
# Build scatter plot for latitude vs temperature
plt.scatter(city_data_pd["Lat"], 
            city_data_pd["Max Temp"],
            edgecolor="black", linewidths=1, marker="o",c = "magenta", 
            alpha=0.8, label="Cities")

# Incorporate the other graph properties
plt.title("City Latitude vs. Max Temperature (%s)" % time.strftime("%x"))
plt.ylabel("Max Temperature (F)")
plt.xlabel("Latitude")
plt.grid(True)
sns.set()

# Save the figure
plt.savefig("Latitude vs Temperature (F).png")

# Show plot
plt.show()

## Latitude vs. Humidity Plot

In [ ]:
# Build the scatter plots for each city types
plt.scatter(city_data_pd["Lat"], 
            city_data_pd["Humidity"],
            edgecolor="black", linewidths=1, marker="o",c = "cyan", 
            alpha=0.8, label="Cities")

# Incorporate the other graph properties
plt.title("City Latitude vs. Humidity (%s)" % time.strftime("%x"))
plt.ylabel("Humidity (%)")
plt.xlabel("Latitude")
plt.grid(True)
sns.set()

# Save the figure
plt.savefig("Latitude vs Humidity (%).png")

# Show plot
plt.show()

## Latitude vs. Cloudiness Plot

In [ ]:
# Build the scatter plots for each city types
plt.scatter(city_data_pd["Lat"], 
            city_data_pd["Cloudiness"],
            edgecolor="black", linewidths=1, marker="o", c = "lime",
            alpha=0.8, label="Cities")

# Incorporate the other graph properties
plt.title("City Latitude vs. Cloudiness (%s)" % time.strftime("%x"))
plt.ylabel("Cloudiness (%)")
plt.xlabel("Latitude")
plt.grid(True)
sns.set()

# Save the figure
plt.savefig("Latitude vs Cloudiness (%).png")

# Show plot
plt.show()

## Latitude vs. Wind Speed Plot

In [ ]:
# Build the scatter plots for each city types
plt.scatter(city_data_pd["Lat"], 
            city_data_pd["Wind Speed"],
            edgecolor="black", linewidths=1, marker="o", c = "orange",
            alpha=0.8, label="Cities")

# Incorporate the other graph properties
plt.title("City Latitude vs. Wind Speed (%s)" % time.strftime("%x"))
plt.ylabel("Wind Speed (mph)")
plt.xlabel("Latitude")
plt.grid(True)
sns.set()

# Save the figure
plt.savefig("Latitude vs Wind Speed (MPH).png")

# Show plot
plt.show()

## Linear Regression

In [ ]:
# OPTIONAL: Create a function to create Linear Regression plots

In [ ]:
# Create Northern and Southern Hemisphere DataFrames
In [1047]: df1, df2 = [x for _, x in df.groupby(df['Sales'] < 30)]


####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression